### 실제로 띄울 맵 코드를 구현하는 공간

In [2]:
import pandas as pd
import folium

# 데이터 읽기
all_farms = pd.read_excel("datas/Windfarm_Data.xlsx")

# 좌표 처리
if "좌표" in all_farms.columns:
    all_farms[["위도_tmp", "경도_tmp"]] = all_farms["좌표"].astype(str).str.split("\t", expand=True)
    all_farms["위도"] = all_farms["위도"].combine_first(all_farms["위도_tmp"]).astype(float)
    all_farms["경도"] = all_farms["경도"].combine_first(all_farms["경도_tmp"]).astype(float)

# NaN 제거
all_farms = all_farms.dropna(subset=["위도", "경도"])

# 지도 생성
m = folium.Map(location=[36, 127], tiles="Cartodb Positron", zoom_start=7)

# 커스텀 아이콘 크기 (픽셀)
icon_size = (10, 10)

# 지도에 커스텀 아이콘과 팝업 추가
for _, farm in all_farms.iterrows():
    # 팝업 HTML
    popup_html = f"""
    <strong>{farm.get('name', '이름 없음')}</strong><br>
    유형: {farm.get('type', '정보 없음')}<br>
    지역: {farm.get('region', '정보 없음')}<br>
    담당처: {farm.get('operator', '정보 없음')}<br>
    발전량: {farm.get('capacity', '정보 없음')} MW
    """
    
    icon = folium.CustomIcon(
        icon_image="datas/dot.png",
        icon_size=icon_size
    )
    
    folium.Marker(
        location=[farm["위도"], farm["경도"]],
        popup=folium.Popup(popup_html, max_width=250),
        icon=icon
    ).add_to(m)

m

m.save("test_map2.html")